# l2-orderbook example usage

In [1]:
import logging
import os

logging.basicConfig(level=os.environ.get("LOGLEVEL", "INFO"))

In [2]:
from scipy import stats

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
import plb

In [5]:
pd.set_option('display.max_colwidth', None)

In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)

In [7]:
%matplotlib inline

In [8]:
INPUT_DIR='/storage/m2/market_data/raw/crypto/binance/BTCUSDT/orderbook'
SAMPLE_DATE='2020-08-01'

In [9]:
def get_binance_orderbook_snap_delta(input_dir, dt):
    '''Load Binance L2 orderbook snapshot and delta files'''
    
    fn_template = os.path.join(input_dir, 'BTCUSDT_T_DEPTH_{}_depth_'.format(dt))
    snap_fn = fn_template + 'snap.csv'
    update_fn = fn_template + 'update.csv'
    
    needed_cols = ['timestamp', 'side', 'price', 'qty'] 
    
    return pd.read_csv(snap_fn, usecols=needed_cols), pd.read_csv(update_fn, usecols=needed_cols)

In [10]:
def get_binance_tops(input_dir, input_date):
    '''Get tops from binance orderbook data and skip over the start-of-day snapshot data'''
    snap, delta = get_binance_orderbook_snap_delta(input_dir, input_date)
    df = pd.concat([snap, delta])

    df = plb.set_types(df)
    tops = plb.get_tops(df)
    
    return tops.iloc[snap.shape[0]:]

In [ ]:
%%time
tops = get_binance_tops(INPUT_DIR, SAMPLE_DATE)

In [ ]:
tops.info()

## EDA

In [ ]:
tops['spread']  = tops['a_0'] - tops['b_0']
tops['mid_price'] = tops['b_0'] + tops['spread']

In [ ]:
tops['b_qty'] = tops[[c for c in tops.columns if c.startswith('bq_')]].sum(axis=1)
tops['a_qty'] = tops[[c for c in tops.columns if c.startswith('aq_')]].sum(axis=1)
tops['tops_imbalance_delta'] = ((tops['b_qty'] - tops['a_qty']) )

In [ ]:
sample = tops.tail(100000)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 3

In [ ]:
ax = (sample['mid_price']/100).plot(label='mid', color='black')

ax2 = ax.twinx()
(sample['tops_imbalance_delta']/1000).ewm(500).mean().plot(ax=ax2, color='green')

ax.figure.legend()
plt.show()